In [ ]:
import requests
from multiprocessing import Process, Manager
import json
import pandas as pd

In [ ]:
def get_story(story_id, stories):
    url = 'https://hacker-news.firebaseio.com/v0/item/%d.json' % story_id
    resp = requests.get(url)
    stories.append(resp.json())

In [ ]:
def get_top_stories():
    manager = Manager()
    stories = manager.list()
    url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    ids = requests.get(url)
    processes = [Process(target=get_story, args=(sid, stories))
                 for sid in ids.json()]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    return stories

In [ ]:
df = pd.read_json('../../data/hn.json')

In [ ]:
df['id'] = df.index.astype(int)

In [ ]:
top_stories = get_top_stories()

In [ ]:
new_df = pd.DataFrame([t for t in top_stories])

In [ ]:
joined = df.append(new_df)

In [ ]:
joined.set_index('id').index

In [ ]:
df = joined.drop_duplicates(subset='id', keep='last')

In [ ]:
df.set_index('id').to_json('../../data/hn.json')